In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn import metrics

In [2]:
# Importamos los datos utilizando pandas
datos=pd.read_csv("./data/data.csv")

# Creo la tabla de candidatos a mano
candidatos=pd.DataFrame(
    [
        [1,'Oscar Andrade', 'Frente Amplio'],
        [2,'Mario Bergara', 'Frente Amplio'],
        [3,'Carolina Cosse', 'Frente Amplio'],
        [4,'Daniel Martínez', 'Frente Amplio'],
        [5,'Verónica Alonso', 'Partido Nacional'],
        [6,'Enrique Antía', 'Partido Nacional'],
        [8,'Carlos Iafigliola', 'Partido Nacional'],
        [9,'Luis Lacalle Pou', 'Partido Nacional'],
        [10,'Jorge Larrañaga', 'Partido Nacional'],
        [11,'Juan Sartori', 'Partido Nacional'],
        [12,'José Amorín', 'Partido Colorado'],
        [13,'Pedro Etchegaray', 'Partido Colorado'],
        [14,'Edgardo Martínez', 'Partido Colorado'],
        [15,'Héctor Rovira', 'Partido Colorado'],
        [16,'Julio María Sanguinetti', 'Partido Colorado'],
        [17,'Ernesto Talvi', 'Partido Colorado'],
        [18,'Pablo Mieres', 'La Alternativa'],
        [19,'Gonzalo Abella', 'Unidad Popular'],        
        [20,'Edgardo Novick', 'Partido de la Gente'],
        [21,'Cèsar Vega', 'PERI'],
        [22,'Rafael Fernández', 'Partido de los Trabajadores'],
        [23,'Justin Graside', 'Partido Digital'],        
        [24,'Gustavo Salle', 'Partido Verde'],
        [25,'Carlos Techera', 'Partido de Todos']
    ],
    columns=['candidatoId','name','party'],
)

datos=datos.merge(candidatos,on=['candidatoId'])

# Sólo por si necesita, cargamos un diccionario con el texto de cada pregunta
preguntas={
    '1': 'Controlar la inflación es más importante que controlar el desempleo. ',
    '2': 'Hay que reducir la cantidad de funcionarios pùblicos',
    '3': 'Deberia aumentar la carga de impuestos para los ricos.',
    '4': 'El gobierno no debe proteger la industria nacional, si las fábricas no son competitivas esta bien que desaparezcan.',
    '5': 'La ley de inclusión financiera es positiva para la sociedad. ',
    '6': 'Algunos sindicatos tienen demasiado poder. ',
    '7': 'Cuanto más libre es el mercado, más libre es la gente. ',
    '8': 'El campo es y debe ser el motor productivo de Uruguay. ',
    '9': 'La inversión extranjera es vital para que Uruguay alcance el desarrollo. ',
    '10': 'Los supermercados abusan del pueblo con sus precios excesivos. ',
    '11': 'Con la vigilancia gubernamental (escuchas telefonicas, e-mails y camaras de seguridad) el que no tiene nada que esconder, no tiene de que preocuparse. ',
    '12': 'La pena de muerte debería ser una opción para los crímenes mas serios. ',
    '13': 'Uruguay debería aprobar más leyes anti corrupción y ser más duro con los culpables. ',
    '14': 'Las FF.AA. deberían tener un rol activo en la seguridad pública. ',
    '15': 'Las carceles deberían ser administradas por organizaciones privadas. ',
    '16': 'Hay que aumentar el salario de los policias significativamente. ',
    '17': 'Para los delitos más graves hay que bajar la edad de imputabilidad a 16 años. ',
    '18': 'Uruguay no necesita un ejército. ',
    '19': 'Uruguay es demasiado generoso con los inmigrantes. ',
    '20': 'La ley trans fue un error. ',
    '21': 'El feminismo moderno no busca la igualdad sino el poder. ',
    '22': 'Para la ley no deberia diferenciarse homicidio de femicidio. ',
    '23': 'La separación de estado y religión me parece importante. ',
    '24': 'La legalización de la marihuana fue un error. ',
    '25': 'La legalización del aborto fue un error. ',
    '26': 'El foco del próximo gobierno debe ser mejorar la educación pública. '
}

# Ordeno los datos por partido y luego por candidato

datos = datos.sort_values(by=['party','name'])

# eliminate candidates with less than 1000 votes
counts_candidate = datos.name.value_counts().reset_index(name="count").query("count > 1000")
counts_candidate.columns = ["name", "count"] 
filtered_data_candidate = datos.merge(counts_candidate, on="name", how="inner")

# eliminate parties with less than 1000 votes
counts_party =  datos.party.value_counts().reset_index(name="count").query("count > 1000")
counts_party.columns = ["party", "count"] 
filtered_data_party = datos.merge(counts_party, on="party", how="inner")


data_candidate = np.array(filtered_data_candidate)
data_party = np.array(filtered_data_party)


In [3]:
filtered_data_party

,id,candidatoId,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,fecha,name,party,count
0,38,3,4,4,3,4,5,5,3,2,...,1,1,5,1,1,5,2019-02-06T16:52:06.318997,Carolina Cosse,Frente Amplio,12421
1,39,3,3,4,5,2,4,4,2,2,...,1,2,5,1,1,3,2019-02-06T17:00:09.220160,Carolina Cosse,Frente Amplio,12421
2,41,3,2,3,4,2,4,4,2,2,...,2,2,5,1,1,4,2019-02-06T17:14:33.621159,Carolina Cosse,Frente Amplio,12421
3,42,3,4,4,3,4,4,4,3,3,...,3,2,5,2,2,4,2019-02-06T17:44:43.482259,Carolina Cosse,Frente Amplio,12421
4,44,3,4,5,3,3,4,5,5,4,...,4,4,5,2,1,5,2019-02-06T18:30:28.400279,Carolina Cosse,Frente Amplio,12421
5,47,3,2,5,5,1,4,5,1,1,...,1,1,5,1,1,5,2019-02-06T22:02:08.552799,Carolina Cosse,Frente Amplio,12421
6,48,3,3,5,5,1,2,5,4,4,...,4,5,5,1,1,5,2019-02-07T03:45:47.511548,Carolina Cosse,Frente Amplio,12421
7,57,3,2,2,5,2,3,2,1,4,...,1,1,5,1,1,4,2019-02-07T15:10:43.354845,Carolina Cosse,Frente Amplio,12421
8,60,3,3,2,5,3,4,3,2,3,...,2,5,5,1,1,5,2019-02-07T15:20:58.874500,Carolina Cosse,Frente Amplio,12421
9,87,3,3,4,5,2,4,4,1,2,...,1,1,5,1,1,5,2019-02-07T17:40:44.657643,Carolina Cosse,Frente Amplio,12421


In [4]:
np.random.shuffle(data_party)
cant_tuples = len(data_party)
amount_training = round(cant_tuples*0.8)
training_party, test_party = data_party[:amount_training,:], data_party[amount_training:,:]

In [5]:
np.random.shuffle(data_candidate)
cant_tuples = len(data_candidate)
amount_training = round(cant_tuples*0.8)
training_candidate, test_candidate = data_candidate[:amount_training,:], data_candidate[amount_training:,:]

## Regresión logistica

In [6]:
penalization_methods = ['l1', 'l2', 'elasticnet', 'none']

In [7]:
# create classfier party

for pen in penalization_methods:
    classifier_party = linear_model.LogisticRegression(penalty=pen, max_iter=10000, tol=0.000001, solver='saga', multi_class='multinomial')
    print("USING " + pen)
    # k fold cross validation training and evaluation
    kf = KFold(n_splits = 5, shuffle=False)
    for train_index, test_index in kf.split(training_party):
        # get the current training and validation
        party_train, party_test = data_party[train_index], data_party[test_index]
        # strip the labels and extra metainfo
        party_train_stripped = party_train[:,2:28]
        party_test_stripped = party_test[:,2:28]
        # get only the labels
        party_train_labels = party_train[:,[30]]
        party_test_labels = party_test[:,[30]]
        # train
        classifier_party.fit(party_train_stripped, party_train_labels.ravel())
        # get the accuracy
        print(classifier_party.score(party_test_stripped, party_test_labels.ravel()))
    print()

USING l1
0.6633583147660522
0.6520153939639457
0.6750405186385737
0.6535656401944895
0.6628849270664505

USING l2
0.6633583147660522
0.6520153939639457
0.6750405186385737
0.6533630470016207
0.6626823338735819

USING elasticnet
0.6633583147660522
0.6520153939639457
0.6750405186385737
0.6533630470016207
0.6626823338735819

USING none
0.6633583147660522
0.6520153939639457
0.6750405186385737
0.6533630470016207
0.6626823338735819



In [9]:
# create classfier candidate

for pen in penalization_methods:
    classifier_candidate = linear_model.LogisticRegression(penalty=pen, max_iter=10000, tol=0.000001, solver='saga', multi_class='multinomial')
    print(classifier_candidate)
    print("USING " + pen)
    # k fold cross validation training and evaluation
    kf = KFold(n_splits = 5, shuffle=False)
    np.random.shuffle(data_candidate)
    for train_index, test_index in kf.split(data_candidate):
        # get the current training and validation
        candidate_train, candidate_test = data_candidate[train_index], data_candidate[test_index]
        # strip the labels and extra metainfo
        candidate_train_stripped = candidate_train[:,2:28]
        candidate_test_stripped = candidate_test[:,2:28]
        # get only the labels
        candidate_train_labels = candidate_train[:,[29]]
        candidate_test_labels = candidate_test[:,[29]]
        # train
        classifier_candidate.fit(candidate_train_stripped, candidate_train_labels.ravel())
        # get the accuracy
        print(classifier_candidate.score(candidate_test_stripped, candidate_test_labels.ravel()))
    print()

TypeError: __init__() got an unexpected keyword argument 'l1_ratio'

## Análisis de componentes principales

In [11]:
party_accum_dict = {'Frente Amplio': 0, 'Partido Nacional': 0, 'Partido Colorado': 0, 'La Alternativa': 0, 'Unidad Popular': 0, 'Partido de la Gente': 0, 'PERI': 0, 'Partido de los Trabajadores': 0, 'Partido Digital': 0, 'Partido Verde': 0, 'Partido de Todos': 0}
party_color_dict = {'Frente Amplio': '#15b01a', 'Partido Nacional': '#0343df', 'Partido Colorado': '#e50000', 'La Alternativa': '#029386', 'Unidad Popular': '#f97306', 'Partido de la Gente': '#ffff15', 'PERI': '#033500', 'Partido de los Trabajadores': '#ceb301', 'Partido Digital': '#0cff0c', 'Partido Verde': '#fe01b1', 'Partido de Todos': '#0485d1'}
for key in party_accum_dict:
    party_accum_dict[key] = len(datos[datos.party == key])
sorted_accum_dict_keys = sorted(party_accum_dict)
for i in range(1, len(sorted_accum_dict_keys)):
    party_accum_dict[sorted_accum_dict_keys[i]] += party_accum_dict[sorted_accum_dict_keys[i - 1]]

In [12]:
respuestas = datos[[str(i) for i in range(1,27)]]

In [13]:
respuestas.head()

,1,2,3,4,5,6,7,8,9,10,...,17,18,19,20,21,22,23,24,25,26
19998,4,4,3,4,5,5,3,2,3,4,...,1,4,1,1,1,1,5,1,1,5
19999,3,4,5,2,4,4,2,2,4,4,...,1,5,2,1,1,2,5,1,1,3
20000,2,3,4,2,4,4,2,2,4,3,...,1,4,2,1,2,2,5,1,1,4
20001,4,4,3,4,4,4,3,3,4,4,...,4,4,2,3,3,2,5,2,2,4
20002,4,5,3,3,4,5,5,4,3,3,...,4,2,3,2,4,4,5,2,1,5


In [14]:
matriz_respuestas = np.matrix(respuestas)
matriz_respuestas = matriz_respuestas.T

Las filas de la matriz son los atributos. Las instancias están por columnas.

### Matriz de covarianza

In [15]:
valor_medio = np.mean(matriz_respuestas, axis=1)
matriz_respuestas_rescalada = matriz_respuestas - valor_medio
matriz_respuestas_rescalada

matrix([[ 0.99096989, -0.00903011, -1.00903011, ..., -0.00903011,
         -0.00903011, -2.00903011],
        [ 0.17903042,  0.17903042, -0.82096958, ..., -1.82096958,
         -0.82096958, -2.82096958],
        [-0.06903566,  1.93096434,  0.93096434, ..., -1.06903566,
          1.93096434,  1.93096434],
        ...,
        [-1.76395352, -1.76395352, -1.76395352, ...,  0.23604648,
         -1.76395352, -1.76395352],
        [-1.02129627, -1.02129627, -1.02129627, ..., -1.02129627,
         -1.02129627, -1.02129627],
        [ 0.52072611, -1.47927389, -0.47927389, ...,  0.52072611,
         -0.47927389,  0.52072611]])

In [16]:
matriz_covarianza = np.cov(matriz_respuestas_rescalada)
valores_propios, vectores_propios = np.linalg.eig(matriz_covarianza)

In [17]:
eigen_pair = [(np.abs(valores_propios[i]), vectores_propios[:,i], i) for i in range(len(valores_propios))]
eigen_pair.sort()
eigen_pair.reverse()

In [18]:
#n is the amount of components
n = 6
matriz_w = np.hstack([(eigen_pair[i][1].reshape(26,1)) for i in range(0,n)])

In [19]:
transformed = np.dot(matriz_respuestas_rescalada.T, matriz_w).T

In [20]:
np.random.shuffle(data_candidate)
cant_tuples = len(data_candidate)
amount_training = round(cant_tuples*0.8)
training_candidate, test_candidate = data_candidate[:amount_training,:], data_candidate[amount_training:,:]

In [ ]:
#Candidate

# Obtain corpus partitions, note this gets all columns
np.random.shuffle(data_candidate)
cant_tuples = len(data_candidate)
amount_training = round(cant_tuples*0.8)
training_candidate, test_candidate = data_candidate[:amount_training,:], data_candidate[amount_training:,:]

# create classifier
classifier_candidate = linear_model.LogisticRegression(penalty='l1', max_iter=10000, tol=0.000001, solver='saga', multi_class='multinomial')

# n is the amount of columns for PCA
for n in range(1, 27):
    matriz_w = np.hstack([(eigen_pair[i][1].reshape(26,1)) for i in range(0,n)])

    transformed = np.dot(matriz_respuestas_rescalada.T, matriz_w).T

    # Create 'filter' array of desired columns
    principal_indices = [eigen_pair[i][2] for i in range(n)]
    indices = np.array([j in principal_indices for j in range(26)])
    indices = np.append([False,False],indices)
    indices = np.append(indices,np.array([False,True,False,False]))
    
    # Apply filter to data
    filtered_columns_training_candidate = training_candidate[:,indices]
    filtered_columns_testing_candidate = test_candidate[:,indices]
    
    print('USING n = ' + str(n) + ':')
    # k fold crossvalidation
    kf = KFold(n_splits = 5, shuffle=False)
    for train_index, test_index in kf.split(filtered_columns_training_candidate):
        # get the current training and validation
        candidate_train, candidate_test = filtered_columns_training_candidate[train_index], filtered_columns_training_candidate[test_index]
        
        # Strip the labels
        candidate_train_stripped = candidate_train[:,0:n]
        candidate_test_stripped = candidate_test[:,0:n]

        # Obtain the labels
        candidate_train_labels = candidate_train[:,[n]]
        candidate_test_labels = candidate_test[:,[n]]
    
        #Train the classifier
        classifier_candidate.fit(candidate_train_stripped, candidate_train_labels.ravel())

        #Evaluate
        
        print(classifier_candidate.score(candidate_test_stripped, candidate_test_labels.ravel()))
    print()

In [ ]:
#%%
#Party

# Obtain corpus partitions, note this gets all columns
np.random.shuffle(data_party)
cant_tuples = len(data_party)
amount_training = round(cant_tuples*0.8)
training_party, test_party = data_party[:amount_training,:], data_party[amount_training:,:]

# create classifier
classifier_party = linear_model.LogisticRegression(penalty='l1', max_iter=1000, tol=0.00001, solver='saga', multi_class='multinomial')

# n is the amount of columns for PCA
for n in range(1, 27):
    matriz_w = np.hstack([(eigen_pair[i][1].reshape(26,1)) for i in range(0,n)])

    transformed = np.dot(matriz_respuestas_rescalada.T, matriz_w).T

    # Create 'filter' array of desired columns
    principal_indices = [eigen_pair[i][2] for i in range(n)]
    indices = np.array([j in principal_indices for j in range(26)])
    indices = np.append([False,False],indices)
    indices = np.append(indices,np.array([False,False,True,False]))
    
    # Apply filter to data
    filtered_columns_training_party = training_party[:,indices]
    filtered_columns_testing_party = test_party[:,indices]
    
    print('USING n = ' + str(n) + ':')
    # k fold crossvalidation
    kf = KFold(n_splits = 5, shuffle=False)
    for train_index, test_index in kf.split(filtered_columns_training_party):
        # get the current training and validation
        party_train, party_test = filtered_columns_training_party[train_index], filtered_columns_training_party[test_index]
        
        # Strip the labels
        party_train_stripped = party_train[:,0:n]
        party_test_stripped = party_test[:,0:n]

        # Obtain the labels
        party_train_labels = party_train[:,[n]]
        party_test_labels = party_test[:,[n]]
    
        #Train the classifier
        classifier_party.fit(party_train_stripped, party_train_labels.ravel())

        #Evaluate
        print(classifier_party.score(party_test_stripped, party_test_labels.ravel()))
    print()

## Evaluación

In [22]:
# PCA n = 25 PARTY.

matriz_w = np.hstack([(eigen_pair[i][1].reshape(26,1)) for i in range(0,25)])

transformed = np.dot(matriz_respuestas_rescalada.T, matriz_w).T

# Create 'filter' array of desired columns
principal_indices = [eigen_pair[i][2] for i in range(25)]
indices = np.array([j in principal_indices for j in range(26)])
indices = np.append([False,False],indices)
indices = np.append(indices,np.array([False,False,True,False]))

# Apply filter to data
filtered_columns_training_party = training_party[:,indices]
filtered_columns_testing_party = test_party[:,indices]

print('evaluating n = 25')

party_train_stripped = filtered_columns_training_party[:,0:25]
party_test_stripped = filtered_columns_testing_party[:,0:25]

# Obtain the labels
party_train_labels = filtered_columns_training_party[:,[25]]
party_test_labels = filtered_columns_testing_party[:,[25]]

#Train the classifier
classifier_party.fit(party_train_stripped, party_train_labels.ravel())

#Evaluate
prediction_labels = classifier_party.predict(party_test_stripped)
accuracy = sklearn.metrics.accuracy_score(party_test_labels, prediction_labels)
precision = sklearn.metrics.precision_score(party_test_labels, prediction_labels,average='macro')
recall = sklearn.metrics.recall_score(party_test_labels, prediction_labels,average='macro')
f_score = sklearn.metrics.f1_score(party_test_labels, prediction_labels,average='macro')
confusion_matrix = sklearn.metrics.confusion_matrix(party_test_labels, prediction_labels, labels=labels)
print(accuracy)
print(precision)
print(recall)
print(f_score)
print(confusion_matrix)

evaluating n = 25


NameError: name 'classifier_party' is not defined

## Clasificar por partido

In [ ]:
# Generate the candidates dictionary
candidates = candidatos.copy()
del candidates['candidatoId']
candidates_dict = {}
for elem in candidates.to_dict('records'):
    candidates_dict[elem['name']] = elem['party']

In [ ]:
# Define the classification function with the candidate classifier
def classify_party(x):
    classified_candidate = classifier_candidate.predict(X)
    return candidates_dict[classified_candidate]

In [ ]:
classified_entries = np.apply_along_axis(classify_party, 0, respuestas)